In [1]:
import os
import time
import json
import pickle
import timeit
import random
import param
import shutil
import collections
import numpy as np
import tensorflow as tf

import stctrain
import datahelper
import stctokenizer
import nuggetdetection as ND
import dialogquality as DQ
import dialogquality_ndfeature as DQNDF
import stcevaluation as STCE

from scipy import stats
from collections import Counter
from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors

import logging
logging.basicConfig(level=logging.ERROR)

Using TensorFlow backend.


In [2]:
doclen = param.doclen
embsize = param.embsize
max_sent = param.max_sent
NDclasses = param.NDclasses
DQclasses = param.DQclasses
sentembsize = param.sentembsize

REMOVE_STOPWORDS = False
TO_LOWER = True
TOKEN_TYPE = 'nltk'
EMB = 'stc' # glove or stc

datahelper = datahelper.DataHelper(embedding_path="../embedding/STCWiki/STCWiki_mincount0.model.bin")
stctokenizer = stctokenizer.STCTokenizer()

## Word Embedding

In [3]:
# corpus = datahelper.prepare_word_embedding_corpus(
#     '../data/text8', 
#     TOKEN_TYPE, 
#     REMOVE_STOPWORDS, 
#     TO_LOWER,
# )

# wordemb_model = Word2Vec(corpus, size=100, min_count=0, workers=4, iter=30, sg=1, window=5)
# word_vectors = wordemb_model.wv

In [4]:
# word_vectors.save("../embedding/STCWiki/STCWiki_mincount0.model.bin")
# datahelper.set_word_vectors(word_vectors)

## Read Corpus & Prepare data for model

In [5]:
# def get_data():
trainX, _, trainND, trainDQ, train_turns, train_masks = datahelper.get_model_train_data(
    'train',
    TOKEN_TYPE, 
    REMOVE_STOPWORDS, 
    TO_LOWER,
    EMB,
    bert=False,
)

devX, _, devND, devDQ, dev_turns, dev_masks = datahelper.get_model_train_data(
    'dev',
    TOKEN_TYPE, 
    REMOVE_STOPWORDS, 
    TO_LOWER,
    EMB,
    bert=False,
)

testX, _, test_turns, test_masks = datahelper.get_model_test_data(
    TOKEN_TYPE, 
    REMOVE_STOPWORDS, 
    TO_LOWER,
    EMB,
    bert=False,
)

testIDs = datahelper.testIDs
trainDQA = [item['A'] for item in trainDQ]
trainDQS = [item['S'] for item in trainDQ]
trainDQE = [item['E'] for item in trainDQ]
devDQA = [item['A'] for item in devDQ]
devDQS = [item['S'] for item in devDQ]
devDQE = [item['E'] for item in devDQ]

dataND = [trainX, trainND, train_turns, train_masks, devX, devND, dev_turns, dev_masks, testX, test_turns, test_masks]
dataDQA = [trainX, trainDQA, train_turns, devX, devDQA, dev_turns, testX, test_turns]
dataDQS = [trainX, trainDQS, train_turns, devX, devDQS, dev_turns, testX, test_turns]
dataDQE = [trainX, trainDQE, train_turns, devX, devDQE, dev_turns, testX, test_turns]

dataDQA_NDF = [trainX, trainDQA, train_turns, trainND, devX, devDQA, dev_turns, devND, testX, test_turns]
dataDQE_NDF = [trainX, trainDQE, train_turns, trainND, devX, devDQE, dev_turns, devND, testX, test_turns]
dataDQS_NDF = [trainX, trainDQS, train_turns, trainND, devX, devDQS, dev_turns, devND, testX, test_turns]

# return dataND, dataDQA, dataDQE, dataDQS, dataDQA_NDF, dataDQE_NDF, dataDQS_NDF, testX, test_turns, test_masks, testIDs

In [6]:
# dataND, dataDQA, dataDQE, dataDQS, dataDQA_NDF, dataDQE_NDF, dataDQS_NDF, testX, test_turns, test_masks, testIDs = get_data()

In [7]:
es = 3
fixed_paramsND  = {
    'epoch':100, 
    'early_stopping':es, 
    'batch_size':30,
    'lr':5e-4,
    'kp':1, 
    'hiddens':1024, 
    'Fsize':[2,3], 
    'gating':False, 
    'bn':True, 
    'method':ND.CNNRNN,
} 

fixed_paramsDQ = {
    'epoch':100, 
    'early_stopping':es, 
    'batch_size':40, 
    'lr':5e-4, 
    'kp':1, 
    'hiddens':1024, 
    'Fsize':[2,2],
    'gating':True, 
    'bn':True, 
}

In [8]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
def show_train_history(title, train, valid, earlystop=es):
    epoch = len(train)
    best = epoch-earlystop
    x = [i for i in range(1, epoch + 1)]
    plt.figure(figsize=(5,12))
    ax = plt.figure().gca()
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.rcParams.update({'font.size': 18})
    plt.plot(x, train, marker='o', linestyle='-', color='b')
    plt.plot(x, valid, marker='o', linestyle='-', color='r')
    plt.axvline(best, color='black')
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
    plt.show()

## Test ND

In [10]:
BEST_PATH = 'ResultPickle/'
# bestND = pickle.load(open(BEST_PATH + 'bestND.p', "rb"))
bestDQAs = pickle.load(open(BEST_PATH + 'memoryDQAs.p', "rb"))
bestDQSs = pickle.load(open(BEST_PATH + 'memoryDQSs.p', "rb"))
bestDQEs = pickle.load(open(BEST_PATH + 'memoryDQEs.p', "rb"))

In [16]:
e = True
for mr in [None]:
    for fn in [[256,512,1024]]:
        for num_layers in [2]:
#                 trainXp = trainX[:int(train_len/10*prop)]
#                 trainNDp = trainND[:int(train_len/10*prop)]
#                 train_turnsp = train_turns[:int(train_len/10*prop)]
#                 train_masksp = train_masks[:int(train_len/10*prop)]
#                 dataND = [trainXp, trainNDp, train_turnsp, train_masksp, devX, devND, dev_turns, dev_masks, testX, test_turns, test_masks]

            testname = 'ND1'
            testND, train_losses, dev_losses = stctrain.start_trainND(
                *dataND, 
                **fixed_paramsND,
                Fnum=fn, num_layers=num_layers, memory_rnn_type=None,
                evaluate=e,
            )

#             show_train_history(testname, train_losses, dev_losses)
            datahelper.pred_to_submission(testND, bestDQAs[0], bestDQSs[0], bestDQEs[0], test_turns, testIDs, filename='{}.json'.format(testname))


x.shape (?, 7, 150, 100)
CNNRNN|13|False|True|2_3|1024|256_512_1024|1|0.0248|0.0939


In [ ]:
# pickle.dump(testND, open('bestND190116.p', 'wb'))

In [ ]:
stop

## Test DQ

In [ ]:
memoryNDs = pickle.load(open('ResultPickle/memoryNDs.p', 'rb'))

In [ ]:
trainX, trainDQA, train_turns

In [ ]:
e = True
method = DQ.CNNCNN
for l in [1]:
    for rm in ['Bi-LSTM']:
        for fn in [[512, 1024]]:
            testname = 'ND_trainsize_{}perc'.format(prop*10)
            print(testname, 'is started')
        
            bestDQA, train_lossesA, dev_lossesA = stctrain.start_trainDQ(
                *dataDQA, 
                **fixed_paramsDQ, scoretype='DQA', method=method,
                Fnum=fn, memory_rnn_type=rm, num_layers=l,
                evaluate=e,
            )
            

            bestDQE, train_lossesE, dev_lossesE = stctrain.start_trainDQ(
                *dataDQE, 
                **fixed_paramsDQ, scoretype='DQE', method=method,
                Fnum=fn, memory_rnn_type=rm, num_layers=l,
                evaluate=e,
            )

            bestDQS, train_lossesS, dev_lossesS = stctrain.start_trainDQ(
                *dataDQS, 
                **fixed_paramsDQ, scoretype='DQS', method=method,
                Fnum=fn, memory_rnn_type=rm, num_layers=l,
                evaluate=e
            )
       
            datahelper.pred_to_submission(memoryNDs[0], bestDQA, bestDQS, bestDQE, test_turns, testIDs, filename='{}.json'.format(testname))
            

In [ ]:
# pickle.dump(bestDQAs, open('memoryDQAs.p', 'wb'))
# pickle.dump(bestDQSs, open('memoryDQSs.p', 'wb'))
# pickle.dump(bestDQEs, open('memoryDQEs.p', 'wb'))

## Test NDF

In [13]:
def submission_to_pred(path='ReTesting/0220_wordemb_test/NoneMemory_3stackCNN_2stackRNN(best).json'):
    import json
    with open(path) as f:
        test_preds_json = json.load(f)
        
    pred = []
    
    for testID in testIDs:
        for test_pred_json in test_preds_json:
            _id = test_pred_json['id']
            if _id != testID:
                continue
            dialogue_nuggets = test_pred_json['nugget']
            dialogue_pred = [] 
            
            for utterance_nugget in dialogue_nuggets:
                utterance_pred = [None] * 7
                if len(utterance_nugget.keys()) == 4:
                    utterance_pred[0] = utterance_nugget['CNUG*']
                    utterance_pred[1] = utterance_nugget['CNUG']
                    utterance_pred[2] = utterance_nugget['CNaN']
                    utterance_pred[3] = utterance_nugget['CNUG0']
                    utterance_pred[4] = 0.
                    utterance_pred[5] = 0.
                    utterance_pred[6] = 0.
                elif len(utterance_nugget.keys()) == 3:
                    utterance_pred[0] = 0.
                    utterance_pred[1] = 0.
                    utterance_pred[2] = 0.
                    utterance_pred[3] = 0.
                    utterance_pred[4] = utterance_nugget['HNUG*']
                    utterance_pred[5] = utterance_nugget['HNUG']
                    utterance_pred[6] = utterance_nugget['HNaN']
                
                dialogue_pred.append(utterance_pred)
                
            while len(dialogue_pred) < 7:
                dialogue_pred.append([0] * 7)
                
            pred.append(dialogue_pred)
    
    return pred

In [14]:
testND = submission_to_pred()

In [15]:
testNDmasked = [np.multiply(nd, mask) for nd, mask in zip(testND, test_masks)]

In [16]:
dataDQA_NDF += [testNDmasked]
dataDQE_NDF += [testNDmasked]
dataDQS_NDF += [testNDmasked]

In [17]:
len(dataDQA_NDF)

11

In [40]:
method = DQNDF.CNNCNN
e = True
testND = np.asarray(testND)

for prop in range(1, 11):
    for mr in ['Bi-LSTM']:
        for fnum in [[512, 1024]]:
            for num_layers in [1]:
                testname = 'DQNDF_trainsize_{}perc'.format(prop*10)
                print(testname, 'is started')
                
                trainXp = trainX[:int(train_len/10*prop)]
                train_turnsp = train_turns[:int(train_len/10*prop)]
                trainNDp = trainND[:int(train_len/10*prop)]
                
                trainDQAp = trainDQA[:int(train_len/10*prop)]      
                dataDQA_NDF = [trainXp, trainDQAp, train_turnsp, trainNDp, devX, devDQA, dev_turns, devND, testX, test_turns, testNDmasked]
                trainDQEp = trainDQE[:int(train_len/10*prop)]         
                dataDQE_NDF = [trainXp, trainDQEp, train_turnsp, trainNDp, devX, devDQE, dev_turns, devND, testX, test_turns, testNDmasked]
                trainDQSp = trainDQA[:int(train_len/10*prop)]              
                dataDQS_NDF = [trainXp, trainDQSp, train_turnsp, trainNDp, devX, devDQS, dev_turns, devND, testX, test_turns, testNDmasked]
                
                
                bestDQNDFA, train_lossesA, dev_lossesA = stctrain.start_trainDQ_NDF(
                    *dataDQA_NDF, 
                    **fixed_paramsDQ, scoretype='DQA', method=method,
                    Fnum=fnum, memory_rnn_type=mr, num_layers=num_layers,
                    evaluate=e,
                )

                bestDQNDFE, train_lossesE, dev_lossesE = stctrain.start_trainDQ_NDF(
                    *dataDQE_NDF, 
                    **fixed_paramsDQ, scoretype='DQE',
                    Fnum=fnum, method=method, memory_rnn_type=mr, num_layers=num_layers,
                    evaluate=e,
                )

                bestDQNDFS, train_lossesS, dev_lossesS = stctrain.start_trainDQ_NDF(
                    *dataDQS_NDF, 
                    **fixed_paramsDQ, scoretype='DQS', 
                    Fnum=fnum, method=method, memory_rnn_type=mr, num_layers=num_layers,
                    evaluate=e
                )

                datahelper.pred_to_submission(testND, bestDQNDFA, bestDQNDFS, bestDQNDFE, test_turns, testIDs, filename='{}.json'.format(testname))

DQNDF_trainsize_10perc is started
CNNCNN|12|True|True|2_2|1024|512_1024|0.12658|0.16478
CNNCNN|14|True|True|2_2|1024|512_1024|0.11628|0.15687
CNNCNN|15|True|True|2_2|1024|512_1024|0.12263|0.16802
DQNDF_trainsize_20perc is started
CNNCNN|12|True|True|2_2|1024|512_1024|0.10614|0.15198
CNNCNN|17|True|True|2_2|1024|512_1024|0.09548|0.14127
CNNCNN|21|True|True|2_2|1024|512_1024|0.10391|0.15318
DQNDF_trainsize_30perc is started
CNNCNN|14|True|True|2_2|1024|512_1024|0.09980|0.14648
CNNCNN|16|True|True|2_2|1024|512_1024|0.09111|0.13518
CNNCNN|8|True|True|2_2|1024|512_1024|0.12357|0.16549
DQNDF_trainsize_40perc is started
CNNCNN|13|True|True|2_2|1024|512_1024|0.08893|0.13366
CNNCNN|15|True|True|2_2|1024|512_1024|0.08648|0.13057
CNNCNN|15|True|True|2_2|1024|512_1024|0.09441|0.14260
DQNDF_trainsize_50perc is started
CNNCNN|15|True|True|2_2|1024|512_1024|0.08880|0.13479
CNNCNN|12|True|True|2_2|1024|512_1024|0.08268|0.12327
CNNCNN|10|True|True|2_2|1024|512_1024|0.09455|0.14574
DQNDF_trainsize_60per

In [ ]:
# pickle.dump(bestDQANDFs, open('memoryDQANDFs.p', 'wb'))
# pickle.dump(bestDQSNDFs, open('memoryDQSNDFs.p', 'wb'))
# pickle.dump(bestDQENDFs, open('memoryDQENDFs.p', 'wb'))